# Szyfrowanie w pełni homomorficzne - kryptosystem BGV

## Pierścienie ilorazowe wielomianów

Obiektem matematycznym powiązanym z ciałami Galois i używanym w kryptografi homomorficznej jest pierścień ilorazowy wielomianów $\mathbb{Z}_q[X]/W(X)$, gdzie $W(X)$ jest danym, konkretnym wielomianem stopnia $n$ a $q$ konkretną liczbą (najczęściej pierwszą).

$\mathbb{Z}_q[X]$ oznacza tutaj pierścień wielomianów dowolnych stopni o współczynnikach będących liczbami z $\mathbb{Z}_q$. Żeby otrzymać reprezentację wielomianu z $\mathbb{Z}[X]$ (tzn. wielomianu o współczynnikach całkowitych) w $\mathbb{Z}_q[X]$ należy obliczyć reprezentację jego współczynników $\mod q$.

Pierścień ilorazowy $\mathbb{Z}_q[X]/W(X)$ to mówiąc prostym językiem pierścień reszt z dzielenia wielomianów z $\mathbb{Z}_q[X]$ przez wielomian $W(X)$, czyli reprezentacją danego wielomianu staje się jego reszta z dzielenia przez $W(X)$.

### Pierścień $\mathbb{Z}_q$ w Sage math:
Konstruujemy pierścień `R=Integers(q)` lub `R=IntegerModRing(q)`, gdzie za `q` podajemy ustaloną liczbę naturalną. Jeżeli chcemy poznać postać liczby `x` w tym pierścieniu, to piszemy `R(x)`. Inną opcją jest funkcja `mod(x,q)`.

In [1]:
R=Integers(3) 
x=R(5)
print(x)
print(type(x))

2
<class 'sage.rings.finite_rings.integer_mod.IntegerMod_int'>


In [2]:
print(4*x,x*4)
print(x**2)
print(x^2)
print(x+4,4+x)

2 2
1
1
0 0


In [3]:
RR=IntegerModRing(3) #to samo, co Integers
x=RR(5)
print(x)
print(type(x))

2
<class 'sage.rings.finite_rings.integer_mod.IntegerMod_int'>


In [4]:
x=mod(5,3) #wynik jest zawsze obiektem klasy IntegerMod_int
print(x)
print(type(x))

2
<class 'sage.rings.finite_rings.integer_mod.IntegerMod_int'>


In [5]:
x**(-1) #można podnosić do ujemnych potęg (o ile podstawa modularna jest liczbą pierwszą)

2

In [6]:
x/2 #dzielenie to po prostu mnożenie przez odwrotność mianownika (o ile podstawa modularna jest l. pierwszą)

1

### Pierścień ilorazowy $\mathbb{Z}_q[X]/W(X)$ w Sage math

Aby utworzyć pierścień ilorazowy $\mathbb{Z}_q[X]/W(X)$ w Sage musimy najpierw utworzyć $\mathbb{Z}_q[X]$, czyli pierścień wielomianów o współczynnikach z $\mathbb{Z}_q$:

`R=PolynomialRing(Integers(q),'X')`

Jeżeli w dalszej części kodu mamy zamiar korzystać z wielomianów z tego pierścienia, to dobrze jest rozdzielić nazewnictwo zmiennych niezależnych

`X=R.gen()`

Teraz każdy wielomian zmiennej `X` będzie przez Sage traktowany jako element pierścienia `R`.

In [7]:
R=PolynomialRing(Integers(5),'X')
X=R.gen()

P=X^6-13*X^4+12*X^2-10*X+6
print(P)
type(P)

X^6 + 2*X^4 + 2*X^2 + 1


<class 'sage.rings.polynomial.polynomial_zmod_flint.Polynomial_zmod_flint'>

Pierścień ilorazowy tworzymy metodą `R.quotient(W,'x')`, gdzie `W` jest dowolnym wielomianem. Podobnie jak poprzednio dobrze jest od razu zdefiniować `x` jako zmienną niezależną wielomianów z nowego pierścienia.

In [8]:
Rq=R.quotient(X^2+X+1,'x')
x=Rq.gen()

p=x^6-13*x^5+12*x^2-10*x+12
print(p)
type(p)

x + 4


<class 'sage.rings.polynomial.polynomial_quotient_ring.PolynomialQuotientRing_field_with_category.element_class'>

## Zadanie 1.

Zaimplementuj pierścień $R_q=\mathbb{Z}_{17}[X]/(X^4+1)$. Sprawdź działanie na poniższych danych testowych.

Dane testowe:

$$w1=7x^6+14x^3$$
$$w2=24x^4-5x^2-7x+13$$
$$w3=23x^5-3x^4+x^3+35x^2+4$$

Reprezentacja w $\mathbb{Z}_{17}[X]/(X^4+1)$:

$$w1=14x^3 + 10x^2$$
$$w2=12x^2 + 10x + 6$$
$$w3=x^3 + x^2 + 11x + 7$$

Arytmetyka:

$$w1+w2=14x^3 + 5x^2 + 10x + 6$$
$$w1\cdot w2=14x^3 + 9x^2 + 2x + 12$$
$$6\cdot w3=6x^3 + 6x^2 + 15x + 8$$
$$w3\cdot 6=6x^3 + 6x^2 + 15x + 8$$

In [9]:
R = PolynomialRing(Integers(17),'X')
X = R.gen()

Rq = R.quotient(X^4+1,'x')
x = Rq.gen()

Reprezentacja:

In [10]:
w1 = 7*x^6 + 14*x^3
w2 = 24*x^4 - 5*x^2 - 7*x + 13
w3 = 23*x^5 - 3*x^4 + x^3 + 35*x^2 + 4

print(w1)
print(w2)
print(w3)

14*x^3 + 10*x^2
12*x^2 + 10*x + 6
x^3 + x^2 + 11*x + 7


Arytmetyka:

In [11]:
print(w1 + w2)
print(w1 * w2)
print(6 * w3)
print(w3 * 6)

14*x^3 + 5*x^2 + 10*x + 6
14*x^3 + 9*x^2 + 2*x + 12
6*x^3 + 6*x^2 + 15*x + 8
6*x^3 + 6*x^2 + 15*x + 8


## Algorytm BGV (Brakerski, Gentry, Vaikuntanathan 2011)

Parametry kryptosystemu:
- $n$ - stopień wielomianu $X^n+1$
- $q$ - podstawa arytmetyki modularnej
- $t$ - podstawa arytmetyki modularnej plaintextu, $t<<q$
- $\chi$ - dyskretny rokład typu Gaussowskiego
- $R_q=\mathbb{Z}_{q}[X]/(X^n+1)$

W uproszczonym modelu kryptosystemu przyjmijmy $n=4$, $q=17$, $t=2$.

`SecretKeyGen(params) -> sk`

- losujemy wektor $s\in\{-1,0,1\}^n$ z *binomial distribution* (prawdopodobieństwo wylosowania 0 jest największe, a prawdopodobieństwa wylosowania -1 i 1 są sobie równe)
- klucz prywatny $sk=s$
    

`PubKeyGen(sk, params) -> (pk0, pk1)`

- losujemy losowy element $a\in R_q$
- losujemy niewielki (w sensie współczynników) błąd $e\in R_q$ z rozkładu $\chi$
- $pk_0=as+te$
- $pk_1=-a$
- klucz publiczny $pk=(pk_0,pk_1)$

`Encrypt(m, pk, params) -> (c0, c1)`

- losujemy niewielkie (w sensie współczynników) błędy $e_0,e_1\in R_q$ z rozkładu $\chi$
- losujemy wektor $u\in\{-1,0,1\}^n$ z *binomial distribution*
- $c_0=pk_0\cdot u+te_0+m$
- $c_1=pk_1\cdot u+te_1$
- szyfrogram $c=(c_0,c_1)$

`Decrypt(c, sk, params)`

- obliczamy $m=c_0+c_1s\mod t$
- zwracamy $m$ jako odszyfrowaną wiadomość

**Podpowiedź:** w praktycznych implementacjach przy deszyfrowaniu nie zwracamy od razu $m=c_0+c_1s\mod t$, tylko robimy "poprawki bit po bicie", tzn. jeżeli $j$-ty bit $m_j$ wyrażenia $m=c_0+c_1s$ jest większy lub równy $\lfloor\frac{q}{t}\rfloor$ to zamiast $m_j\mod t$ obliczamy "poprawiony bit" $m_j-q\mod t$.

## Zadanie 2.

Zaimplementuj powyższą uproszczoną wersję algorytmu BGV, najlepiej w dwóch krokach:
- wstępna implementacja bez generatorów losowych, z ustalonymi wartościami $sk,e,a,e_0,e_1,u$. Sprawdź jej działanie na danych testowych.
- pełna wersja kryptosystemu z $sk,e,a,e_0,e_1,u$ generowanymi z odpowiednich rozkładów. Sprawdź poprawność działania dla 100 losowych wiadomości $m$.
- w celu poprawy poprawności deszyfrowania można w funkcji `Decrypt` sprawdzać współczynniki wielomianu $c_0+c_1s$ pojedynczo i jeżeli któryś przekracza $\lfloor\frac{q}{t}\rfloor$ (co na pewno spowoduje błąd deszyfrowania) to przed operacją $\mod t$ odjąć od niego $q$


Dane testowe 1:
- $sk= [ 0,  0, 16,  0]$
- $e= [16,  0, 16,  0]$
- $pk_0= [14, 10 ,15, 11]$
- $pk_1= [ 6, 16, 10,  0]$
- $m=[1, 0, 1, 0]$
- $e_0= [0, 0, 1, 0]$
- $e_1= [ 0, 16,  0, 16]$
- $u= [ 1,  0, 16,  0]$
- $c_0= [ 2,  5, 16, 16]$
- $c_1= [ 1, 16,  1, 11]$

Dane testowe 2:
- $sk= [0, 1, 0, 0]$
- $e= [1, 0, 0, 0]$
- $pk_0= [15,  6, 15, 13]$
- $pk_1= [15, 13,  4, 11]$
- $m=[0, 1, 1, 1]$
- $e_0= [1, 0, 0, 0]$
- $e_1= [0, 0, 1, 0]$
- $u= [ 0,  0,  0, 16]$
- $c_0= [ 4, 12,  3,  5]$
- $c_1= [ 2,  4, 15,  6]$


In [12]:
# Ring parameters
n, q, t = 4, 257, 2
R.<x> = PolynomialRing(Zmod(q))
Rq = R.quotient(x^n + 1, 'xbar')
threshold = q // t

#### Implementacja 1

Wstępna wersja dla podanych danych testowych

In [13]:
def SecretKeyGen_fixed(values):
    """Return a fixed secret key polynomial."""
    sk = Rq(values)
    return sk

def PubKeyGen_fixed(sk, a_vals, e_vals):
    """Return (pk0, pk1) for fixed a and e."""
    a = Rq(a_vals)
    e = Rq(e_vals)
    pk0 = a * sk + t * e
    pk1 = -a
    return pk0, pk1

def Encrypt_fixed(m_vals, pk, u_vals, e0_vals, e1_vals):
    """Encrypt a fixed message vector using fixed u, e0, e1."""
    u  = Rq(u_vals)
    e0 = Rq(e0_vals)
    e1 = Rq(e1_vals)
    m  = Rq(m_vals)
    c0 = pk[0] * u + t * e0 + m
    c1 = pk[1] * u + t * e1
    return c0, c1

def Decrypt(ct, sk):
    c0, c1 = ct
    raw = (c0 + c1 * sk).lift()
    bits = []
    for coeff in raw.list():
        c = int(coeff) % q
        if c >= threshold:
            c -= q
        bits.append(c % t)
    if len(bits) < n:
        bits.extend([0] * (n - len(bits)))
    return bits[:n]

In [14]:
def run_case(label, *, sk, a, e, m, u, e0, e1):
    sk_poly = SecretKeyGen_fixed(sk)
    pk      = PubKeyGen_fixed(sk_poly, a, e)
    ct      = Encrypt_fixed(m, pk, u, e0, e1)
    dec     = Decrypt(ct, sk_poly)
    status  = "PASS" if dec == m else "FAIL"
    print(f"{label}: decrypted {dec}  expected {m}  → {status}")
    return dec == m

print("Deterministic vectors")
ok1 = run_case(
    "Test 1",
    sk =[0, 0, 16, 0],
    a  =[6, 16, 10, 0],
    e  =[16, 0, 16, 0],
    m  =[1, 0, 1, 0],
    u  =[1, 0, 16, 0],
    e0 =[0, 0, 1, 0],
    e1 =[0, 16, 0, 16]
)

ok2 = run_case(
    "Test 2",
    sk =[0, 1, 0, 0],
    a  =[15, 13, 4, 11],
    e  =[1, 0, 0, 0],
    m  =[0, 1, 1, 1],
    u  =[0, 0, 0, 16],
    e0 =[1, 0, 0, 0],
    e1 =[0, 0, 1, 0]
)

print("\nOverall deterministic result:",
      "PASS" if ok1 and ok2 else "FAIL")

Deterministic vectors
Test 1: decrypted [1, 0, 1, 0]  expected [1, 0, 1, 0]  → PASS
Test 2: decrypted [0, 1, 1, 1]  expected [0, 1, 1, 1]  → PASS

Overall deterministic result: PASS


#### Implementacja 2

In [15]:
from sage.stats.distributions.discrete_gaussian_integer import DiscreteGaussianDistributionIntegerSampler

binomial_sampler = GeneralDiscreteDistribution([0.25, 0.50, 0.25])

def sample_poly():
    coeffs = [binomial_sampler.get_random_element() - 1 for _ in range(n)]
    return Rq(coeffs)

sigma = 0.3
gaussian_sampler = DiscreteGaussianDistributionIntegerSampler(sigma)

def noise_poly():
    coeffs = [gaussian_sampler() for _ in range(n)]
    return Rq(coeffs)

def uniform_poly():
    return Rq.random_element()

def SecretKeyGen():
    sk = sample_poly()
    return sk

def PubKeyGen(sk):
    a = uniform_poly()
    e = noise_poly()
    pk0 = a * sk + t * e
    pk1 = -a
    return pk0, pk1

def Encrypt(message_bits, pk):
    m  = Rq(message_bits)
    u  = sample_poly()
    e0 = noise_poly()
    e1 = noise_poly()
    c0 = pk[0] * u + t * e0 + m
    c1 = pk[1] * u + t * e1
    return c0, c1

In [16]:
trials = 10000
success = 0

for _ in range(trials):
    sk = SecretKeyGen()
    pk = PubKeyGen(sk)
    msg = [randint(0, t - 1) for _ in range(n)]
    ct  = Encrypt(msg, pk)
    if Decrypt(ct, sk) == msg:
        success += 1

rate = 100.0 * success / trials
print(f'Random-variant success rate: {success}/{trials}  =  {rate:.2f}%')

Random-variant success rate: 10000/10000  =  100.00%


## Operacje homomorficzne na szyfrogramach

### Dodawanie `Add`

Załóżmy, że mamy dwie wiadomości zaszyfrowane tym samym kluczem prywatnym, tzn. dwie pary $(c_0,c_1)$ oraz $(c'_0,c'_1)$. Naturalnym sposobem zdefiniowania sumy jest $$c_0^{\ast}=c_0+c'_0$$ $$c_1^{\ast}=c_1+c'_1$$czyli szyfrogram sumy to $$c^{\ast}=(c_0^{\ast},c_1^{\ast}).$$

To podejście me jeden problem: z każdą operacją sumowania wzrasta zaszumienie końcowego szyfrogramu, co może skutkować błędnym deszyfrowaniem, jednak nie aż tak bardzo jak to się dzieje w przypadku mnożenia.

### Mnożenie `Mul`

Jak przy dodawaniu mamy dwie wiadomości zaszyfrowane tym samym kluczem prywatnym, tzn. dwie pary $(c_0,c_1)$ oraz $(c'_0,c'_1)$. W przypadku naturalnej definicji mnożenia sprawy się komplikują: jeżeli popatrzymy na funkcję `Decrypt`, to wiadomości $m$ i $m'$ kryjące się za naszymi szyfrogramami są postaci $$m=c_0+c_1s$$ $$m'=c'_0+c'_1s.$$Jeżeli teraz pomnożymy te dwie wiadomości, to otrzymamy $$mm'=(c_0+c_1s)(c'_0+c'_1s)=c_0c'_0+(c_0c'_1+c'_0c_1)s+c_1c'_1s^2$$
Otrzymujemy zatem **trzy** współrzędne końcowego szyfrogramu:
\begin{eqnarray*}
c^{\ast}_0=c_0c'_0\\
c^{\ast}_1=c_0c'_1+c'_0c_1\\
c^{\ast}_2=c_1c'_1
\end{eqnarray*}
Jako wynik mnożenia zwracamy szyfrogram $$c^{\ast}=(c_0^{\ast},c_1^{\ast},c_2^{\ast}).$$
W tym przypadku oprócz problemu z narastającym zaszumieniem mamy jeszcze problem z dodatkową współrzędną, której nie bierze pod uwagę nasza implementacja funkcji deszyfrującej.

### Prosta relinearyzacja `KeySwitch`

Niech $c^{\ast}=(c_0^{\ast},c_1^{\ast},c_2^{\ast})$ będzie wynikiem mnożenia dwóch wiadomości $m_1$ i $m_2$ zaszyfrowanych przy pomocy klucza publicznego $(pk_0, pk_1)$ i klucza prywatnego $s$. Żeby pozbyć się współrzędnej $c_2^{\ast}$ (i przekształcić postać iloczynu $mm'$ z kwadratowej na liniową) stosujemy *zmianę klucza*.

**Krok 1 - rozkład wielomianu.** Najpierw zapisujemy wszystkie współczynniki wielomianu $c_2^{\ast}=w_0+w_1x+w_2x^2+w_3x^3$ w reprezentacji w systemie dwójkowym, tzn. $$w_i=\sum_{j=0}^{\lfloor \log_2 q\rfloor+1}2^jw^{(j)}_i,\ \ i=0,1,2,3$$

Konstruujemy nowe wielomiany dla $j=0,...,\lfloor \log_2 q\rfloor+1$ $$c_2^{\ast (j)}=w^{(j)}_0+w^{(j)}_1x+w^{(j)}_2x^2+w^{(j)}_3x^3$$i za ich pomocą rozkładamy wielomian $c_2^{\ast}$ $$c_2^{\ast}=\sum_{j=0}^{\lfloor \log_2 q\rfloor+1}2^j c_2^{\ast (j)}\mod q$$

**Krok 2 - generowanie wskazówek.** Dla $j=0,...,\lfloor \log_2 q\rfloor+1$ z klucza prywatengo $s$ generujemy tzw. *wskazówki*: $$(ek_0^{(j)},ek_1^{(j)})=(a_js+te_j+2^js^2,-a_j),$$gdzie $a_j\in R_q$ są generowane losowo z rozkładu jednostajnego a błedy $e_i\in R_q$ - losowo z rozkładu typu Gaussowskiego (jak przy generowaniu kluczy).

**Krok 3 - nowy szyfrogram.** Generujemy nowy szyfrogram $(\widehat{c}_0,\widehat{c}_1)$: $$\widehat{c}_0=c_0^{\ast}+\sum_{j=0}^{\lfloor \log_2 q\rfloor+1}ek_0^{(j)}c_2^{\ast (j)}$$
$$\widehat{c}_1=c_1^{\ast}+\sum_{j=0}^{\lfloor \log_2 q\rfloor+1}ek_1^{(j)}c_2^{\ast (j)}$$

Po zdeszyfrowaniu $(\widehat{c}_0,\widehat{c}_1)$ z kluczem $s$ powinniśmy otrzymać wiadomość będącą wynikiem mnożenia dwóch wiadomości $m_1$ i $m_2$.

## Zadanie 3.

Zaimplementuj funkcje `Add`, `KeySwitch` oraz `Mul` realizujące powyższe algorytmy.
- sprawdź działanie dodawania i mnożenia dla jednej operacji ($m_1+m_2, m_1*m_2$). Pamiętaj o wykorzystaniu funkcji `KeySwitch` przy mnożeniu. Dobierz parametry kryptosystemu tak, żeby po deszyfrowaniu otrzymać poprawne wyniki.
- ile operacji dodawania możemy wykonać zanim narastające błędy spowodują błędne deszyfrowanie ($m_1+m_2+m_3+...$)?
- a ile operacji mnożenia ($m_1*m_2*m_3*...$)?
- sprawdź jak wygląda skuteczność deszyfrowania w przypadku mieszania operacji, np. $m_1*m_2+m_3$. Dla jakiej głębokości $N$ operacji mieszanych na wiadomościach deszyfrowanie jest poprawne?

Przez głębokość $N$ operacji mieszanych rozumiemy kombinację postaci: iloczyn $N$ wiadomości plus iloczyn $N-1$ wiadomości plus iloczyn $N-2$ wiadomości plus ... plus iloczyn dwóch wiadomości plus jedna wiadomość.

Otrzymane rezultaty (maksymalna głębokość operacji dodawania, operacji mnożenia i operacji mieszanych) opisz pełnym zdaniem w osobnej komórce pod testami.

In [17]:
n, q, t = 4, 999_983, 2           # or any large prime
R.<x> = PolynomialRing(Zmod(q))
Rq = R.quotient(x^n + 1, 'xbar')
threshold = q // t

In [18]:
from math import floor, log2

def small_poly():
    return Rq([binomial_sampler.get_random_element() - 1 for _ in range(n)])

def Add(ct1, ct2):
    c0a, c1a = ct1
    c0b, c1b = ct2
    return c0a + c0b, c1a + c1b

def RelinearizationKey(sk):
    s2 = sk * sk
    L  = floor(log2(q)) + 1
    rk = []
    for j in range(L):
        a = uniform_poly()
        e = small_poly()
        rk.append((a * sk + (2 ** j) * s2 + t * e, -a))
    return rk

def KeySwitch(ct3, rk):
    c0, c1, c2 = ct3
    c0_new, c1_new = c0, c1
    for j, (ek0, ek1) in enumerate(rk):
        bits = [(int(z) >> j) & 1 for z in c2.list()]
        bpoly = Rq(bits)
        c0_new += ek0 * bpoly
        c1_new += ek1 * bpoly
    return c0_new, c1_new

def Mul(ct1, ct2):
    c0a, c1a = ct1
    c0b, c1b = ct2
    return (
        c0a * c0b,
        c0a * c1b + c1a * c0b,
        c1a * c1b
    )

In [19]:
def test_depths(max_add_terms=10_000):
    sk = SecretKeyGen()
    pk = PubKeyGen(sk)
    rk = RelinearizationKey(sk)

    add_ct = Encrypt([0,0,0,0], pk)
    add_val = 0
    add_depth = 0
    for _ in range(max_add_terms):
        add_ct = Add(add_ct, Encrypt([1,0,0,0], pk))
        add_val = (add_val + 1) % 2
        if Decrypt(add_ct, sk)[0] == add_val:
            add_depth += 1
        else:
            break

    mul_ct = Encrypt([1,0,0,0], pk)
    mul_depth = 0
    while True:
        mul_ct = KeySwitch(Mul(mul_ct, Encrypt([1,0,0,0], pk)), rk)
        if Decrypt(mul_ct, sk)[0] == 1:
            mul_depth += 1
        else:
            break

    N = 1
    prod_ct = Encrypt([1,0,0,0], pk)
    sum_ct  = prod_ct
    expected = 1
    while True:
        prod_ct  = KeySwitch(Mul(prod_ct, Encrypt([1,0,0,0], pk)), rk)
        sum_ct   = Add(sum_ct, prod_ct)
        N       += 1
        expected = (expected + 1) % 2
        if Decrypt(sum_ct, sk)[0] != expected:
            N -= 1
            break
    return add_depth, mul_depth, N

# sanity-check: one add, one mul
sk_chk = SecretKeyGen()
pk_chk = PubKeyGen(sk_chk)
a_ct   = Encrypt([1,0,0,0], pk_chk)
b_ct   = Encrypt([0,0,0,0], pk_chk)
assert Decrypt(Add(a_ct, b_ct), sk_chk)[0] == 1
rk_chk = RelinearizationKey(sk_chk)
prod_rl = KeySwitch(Mul(a_ct, b_ct), rk_chk)
assert Decrypt(prod_rl, sk_chk)[0] == 0

# depth results
add_d, mul_d, mix_d = test_depths(1_000_000)

print(f"Maksymalna liczba dodawań        : {add_d}")
print(f"Maksymalna liczba mnożeń         : {mul_d}")
print(f"Maksymalna głębokość mieszana N  : {mix_d}")
print(
    f"\nDla wybranych parametrów poprawnie deszyfrujemy "
    f"{add_d} dodawań, {mul_d} mnożeń oraz operacje mieszane do głębokości N = {mix_d}."
)

Maksymalna liczba dodawań        : 500024
Maksymalna liczba mnożeń         : 131
Maksymalna głębokość mieszana N  : 110

Dla wybranych parametrów poprawnie deszyfrujemy 500024 dodawań, 131 mnożeń oraz operacje mieszane do głębokości N = 110.
